In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pickle

In [2]:
# Загружаем стоп слова для русского и английского языка
stop1 = list(stopwords.words('english'))
stop2 = list(stopwords.words('russian'))

stop3 = ['africa', 'african', 'afrika', 'antigua', 'arab', 'arabia', 'arabiyyah', 'art', 'aş', 'bahamas', 'bahrayn', 'barbuda', 'belgien', 'belgique', 'belgië', 'bielaruś', 'bissau', 'bosnia', 'bukchosŏn', 'bulgariya', 'burkina', 'cabo', 'cape', 'centrafricaine', 'central', 'channel', 'chosŏn', 'città', 'city', 'comores', 'costa', 'crna', 'côte', 'dawlat', 'del', 'democratic', 'dhivehi', 'dominican', 'dominicana', 'dr', 'druk', 'du', 'démocratique', 'east', 'ecuatorial', 'el', 'emirates', 'equatorial', 'faeroe', 'faso', 'federated', 'french', 'gabonaise', 'gora', 'grenadines', 'guiana', 'helena', 'hellas', 'hercegovína', 'herzegovina', 'holy', 'hong', 'ia', 'imārat', 'islands', 'isle', 'ityop', 'ivoire', 'kingdom', 'kitts', 'kong', 'korea', 'koromi', 'kuwayt', 'kıbrıs', 'lanka', 'lankā', 'leone', 'leste', 'lester', 'lucia', 'macedonia', 'makedonija', 'man', 'marino', 'marshall', 'maɣréb', 'micronesia', 'mongol', 'mueang', 'nam', 'nevis', 'new', 'north', 'papua', 'principe', 'raajje', 'república', 'rica', 'république', 'sahara', 'saint', 'sak', 'salvador', 'san', 'sao', 'saudi', 'see', 'severna', 'sierra', 'solomon', 'soomaaliya', 'south', 'srbija', 'sri', 'state', 'states', 'suid', 'são', 'thai', 'timor', 'tobago', 'tome', 'tomé', 'trinidad', 'uburundi', 'ul', 'uls', 'umān', 'united', 'urdun', 'vatican', 'vaticano', 'velo', 'verde', 'vincent', 'việt', 'western', 'yaman', 'yul', 'zbekiston', 'zealand', 'şūmāl', 'ūdiyyah', 'ελλάς', 'κύπρος', 'беларусь', 'србија']

stop = stop1 + stop2 + stop3


lemmatizer = WordNetLemmatizer()

# Функция для предобработки. Удаление знаков пуктуации, приведение к нижнему регистру, лемматизация.
def preprocessing(line):
    line = line.lower()
    line = re.sub(r'[.,"\'-?:!;]', "", line)
    line = nltk.word_tokenize(line)
    line = ' '.join([lemmatizer.lemmatize(w) if w not in stop_words else "" for w in line])
    return line


# Добавим в стоп слова названия стран и слова-формы собственности организаций.
df_stop = pd.read_csv('stop_words.csv')
df_countries = pd.read_csv('stop_countries.csv', header=None)
stop_company = list(df_stop['0'])
stop_countries = list(df_countries[1])
stop_words = list(set(stop+ stop_company + stop_countries))

In [3]:
# Читаем наш датасет.
df = pd.read_csv('train.csv')

In [4]:
# Посмотрим на данные.
df.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


In [5]:
# Объединим два столбца, посмотрим сколько всего названий.
text = list(df['name_1']) + list(df['name_2'])

print(len(text))

995638


In [6]:
# Оставим только уникальные названия.

text_unic = list(set(text))


text_unic_preproc = [preprocessing(name) for name in text_unic]

my_vectorizer = pickle.load(open('vectorizer_2.pickle','rb'))

# Функция для создания эмбеддингов из названий компаний.
def sentence_embedding_fl(string, vectorizer):
    tf_vectorizer = vectorizer
    sentence_list = [tf_vectorizer.transform([word]).toarray() for word in string.split()]
        
    result = np.sum(np.array(sentence_list), axis=0)
    if len(result.shape) == 0:
        result = np.zeros((1,9776))
    
    return result

name_embeddings = [sentence_embedding_fl(name, my_vectorizer) for name in  text_unic_preproc]

print(len(text_unic))
print(len(text_unic_preproc))
print(len(name_embeddings))

18022
18022
18022


In [7]:
pickle.dump(name_embeddings, open('name_embeddings.pickle', "wb"))

pickle.dump(text_unic, open('companies_name_unic.pickle', "wb"))